# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298392392526                   -0.85    4.4
  2   -8.300195008645       -2.74       -1.25    1.0
  3   -8.300441856838       -3.61       -1.87    4.1
  4   -8.300462466375       -4.69       -2.75    1.6
  5   -8.300464425571       -5.71       -3.08    4.0
  6   -8.300464522614       -7.01       -3.22    3.8
  7   -8.300464580054       -7.24       -3.37    3.2
  8   -8.300464611878       -7.50       -3.50    1.8
  9   -8.300464633641       -7.66       -3.68    1.0
 10   -8.300464638154       -8.35       -3.77    7.5
 11   -8.300464642588       -8.35       -3.98    1.0
 12   -8.300464643982       -8.86       -4.55    1.9


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64196190762                   -0.70    5.5
  2   -16.67869906961       -1.43       -1.14    1.5
  3   -16.67923053505       -3.27       -1.87    3.9
  4   -16.67928185901       -4.29       -2.77    3.4
  5   -16.67928614834       -5.37       -3.13    5.0
  6   -16.67928620561       -7.24       -3.53    1.2
  7   -16.67928622086       -7.82       -4.02    1.4


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32552462442                   -0.56    6.1
  2   -33.33270753227       -2.14       -1.00    1.0
  3   -33.33502792154       -2.63       -1.69    4.1
  4   -33.33683951196       -2.74       -2.30    3.4
  5   -33.33693328963       -4.03       -2.77   12.1
  6   -33.33694380448       -4.98       -3.68    3.6
  7   -33.33694386627       -7.21       -4.02    8.2


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298433439156                   -0.85    4.6
  2   -8.300270781478       -2.74       -1.59    1.0
  3   -8.300457572988       -3.73       -2.63    5.9
  4   -8.300419556168   +   -4.42       -2.43   16.9
  5   -8.300464377744       -4.35       -3.57    2.5
  6   -8.300464577394       -6.70       -3.89    4.5
  7   -8.300464643359       -7.18       -4.83    3.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31594061586                   -0.56    6.2
  2   -33.20176291461   +   -0.94       -1.20    1.6
  3   +53.47316167350   +    1.94       -0.10   10.1
  4   -33.22901152644        1.94       -1.35    8.8
  5   -33.23821834534       -2.04       -1.33    4.5
  6   -29.60685804062   +    0.56       -0.78    4.6
  7   -33.26500285224        0.56       -1.61    6.5
  8   -33.33475107654       -1.16       -2.17    3.2
  9   -33.33553803200       -3.10       -2.20    3.2
 10   -33.33639667690       -3.07       -2.49    2.2
 11   -33.33678743120       -3.41       -2.80    2.0
 12   -33.33679512296       -5.11       -2.87    1.9
 13   -33.33691860369       -3.91       -3.22    3.1
 14   -33.33693866673       -4.70       -3.63    3.0
 15   -33.33694195176       -5.48       -3.84    3.5
 16   -33.33694381728       -5.73       -4.32    3.1


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.